In [1]:
def check_NA(response):
    apologetic_flag = True if 'sorry' in response or 'does not have the necessary detail' in response or 'cannot accurately identify' in response or 'Sorry' in response or "can't provide" in response or "cannot provide" in response or "does not provide" in response or "cannot assist" in response or "not possible" in response or "unable" in response or 'unknown' in response or 'not clear' in response or 'not determined' in response or 'not known' in response or 'not provided' in response or 'not mentioned' in response or 'not given' in response or 'not specified' in response or 'may not be visible' in response or 'not visible' in response or 'unfortunately' in response or 'not clearly visible' in response else False
    return apologetic_flag

def remove_verbose(response, organism='fish'):
    response = response.replace('The answer is: ', '')
    response = response.replace(f'The {organism} in the image is a ', '')
    response = response.replace(f'The {organism} in the image is called', '')
    response = response.replace(f'The {organism} in the photo is a', '')
    response = response.replace(f'The species and scientific name of the {organism} in the image are', '')
    response = response.replace(f'The species in the image is', '')
    response = response.replace(f'The species of the {organism} in the image is', '')
    response = response.replace(f'The scientific name of the {organism} in the image is', '') 
    response = response.replace(f'The species of the {organism} in the picture is', '')
    response = response.replace(f'The species of the {organism} in the photo is', '')
    response = response.replace(f'The species of the {organism} in this picture is','')
    response = response.replace(f'The {organism} in the photo is called', '')
    return response.strip()




def classification_eval_selection(data, model_name, organism='fish', task_type='direct'):
    out_dict = {
        'Model': model_name,
        'Question-type': task_type,
        'correct_count': 0,
        'correct_ge_count' : 0, # only genus correct, partial
        'NA_count' : 0,
        'incorrect_count': 0,
    }
    
    for data_idx in range(len(data)):
        data_item = data[data_idx]
        
        gt_sci_name = data_item['target-class']
        if gt_sci_name != gt_sci_name:
            continue
        gt_genus = gt_sci_name.split(' ')[0]

        if len(gt_sci_name) == 2:
            gt_species = gt_sci_name.split(' ')[1]

        response = data_item['output']
    
        response = remove_verbose(response, organism=organism)

        gt_option = data_item['option-gt']
        
        # print(f'Ground-truth:{gt_option}', response)

        if len(response)==1:
            if gt_option.lower() == response.lower():
                out_dict['correct_count'] += 1
            else:
                out_dict['incorrect_count'] += 1
            continue

        elif response.lower() in ['a)', 'b)', 'c)', 'd)']:
            if gt_option.lower() == response.lower()[0]:
                out_dict['correct_count'] += 1
            else:
                out_dict['incorrect_count'] += 1
            continue
                
        # correct
        if gt_sci_name.lower() in response.lower():
            out_dict['correct_count'] += 1
            continue

        # partial, correct genus only
        if gt_genus.lower() in response.lower():
            out_dict['correct_ge_count'] += 1
            continue

        if check_NA(response.lower()):
            out_dict['NA_count'] += 1
            continue

        out_dict['incorrect_count'] += 1

    total = out_dict['correct_count'] + out_dict['correct_ge_count'] + out_dict['NA_count'] + out_dict['incorrect_count']


    percnt_out_dict = {
        'Model': model_name,
        'Question-type': task_type,
        'correct_count': out_dict['correct_count']*100/total if total!=0 else 0,
        'correct_ge_count' : out_dict['correct_ge_count']*100/total if total!=0 else 0,
        'NA_count' : out_dict['NA_count']*100/total if total!=0 else 0,
        'incorrect_count': out_dict['incorrect_count']*100/total if total!=0 else 0,
    }
    print(f'Evaluation completed for {model_name}.')

    return out_dict, percnt_out_dict


In [2]:
import pandas as pd
import json

PROMPT_MODELS = ['gpt-4v', 'gpt-4o', 'llava-v1.5-7b', 'llava-v1.5-13b', 'cogvlm-chat', 'blip-flan-xl', 'blip-flan-xxl']
# MODEL_NAMES = ['gpt-4v',
#                'gpt-4o',
#                'llava-v1.5-7b',
#                'llava-v1.5-13b',
#                'cogvlm-chat',
#                'blip-flan-xl',
#                'blip-flan-xxl',
#                'minigpt4-vicuna-7B',
#                'minigpt4-vicuna-13B',
#                'instruct-flant5xl',
#                'instruct-flant5xxl',
#                'instruct-vicuna7b',
#                'instruct-vicuna13b']
# PROMPT_MODELS = MODEL_NAMES

def classification_tables(filename, model_lists=PROMPT_MODELS, full_display=False):

    # Initialize empty lists to store data
    model_list = []
    question_type_list = []
    correct_count_list = []
    correct_ge_count_list = []
    na_count_list = []
    incorrect_count_list = []
    
    # Read the JSON lines file line by line
    with open(filename, 'r') as file:
        for line in file:
            data = json.loads(line)
            if data['Model'] not in model_lists:
                continue
            model_list.append(data['Model'])
            question_type_list.append(data['Question-type'])
            correct_count_list.append('{:.2f}'.format(data['correct_count']))
            correct_ge_count_list.append('{:.2f}'.format(data['correct_ge_count']))
            na_count_list.append('{:.2f}'.format(data['NA_count']))
            incorrect_count_list.append('{:.2f}'.format(data['incorrect_count']))
    
    # Create a DataFrame
    df = pd.DataFrame({
        'Models': model_list,
        'Question-type': question_type_list,
        'correct_count': correct_count_list,
        'correct_ge_count': correct_ge_count_list,
        'NA_count': na_count_list,
        'incorrect_count': incorrect_count_list
    })
    
    # Display the DataFrame
    if full_display==False:
        display(df)

    return df[['correct_count']]

In [3]:
def write_evaluation(dataset, task_type, task, organism, start_dir, prompting_technique, model_list = None):

    model_list = MODEL_NAMES if model_list == None else model_list
    
    for model_name in model_list:
        datalist = []
        
        for chunkid in range(10):
            if organism == 'fish':
                some_value = 50
            elif organism == 'bird':
                some_value = 50
            elif organism == 'butterfly':
                some_value = 50
                
            RESULT_FILE_TEMPT = f'{start_dir}/{dataset}/{prompting_technique}/{task}/{task_type}/{task}_{model_name}_{task_type}_num_{some_value}_chunk_{chunkid}.jsonl'
            datalist+=read_file(RESULT_FILE_TEMPT)
    
        print(f'Starting Evaluations for model {model_name} on {len(datalist)} results.')
    
        if task_type == 'direct':
            _, percent_result_dict = classification_eval_direct(data=datalist, model_name=model_name, task_type=task_type, organism=organism)
        elif task_type == 'selection':
            _, percent_result_dict = classification_eval_selection(data=datalist, model_name=model_name, task_type=task_type, organism=organism)

        
        writer = jsonlines.open(f'./tables/classification_{dataset}_{prompting_technique}_{task_type}_{organism}.jsonl', mode='a')
        writer.write(percent_result_dict)
        writer.close()

In [4]:
import os
import json
import jsonlines

def read_file(filepath):
    if os.path.exists(filepath) == False:
        print(f'{filepath} not found.')
        return []

    datalist = []

    with open(filepath, "r", encoding='utf-8') as fh:
        for line in fh.readlines():
            if line:
                try: # if the response writing creates any issue, this try-catch will handle it.
                    dict_ = json.loads(line) 
                except:
                    continue
                datalist.append(dict_)

    return datalist

In [5]:
MODEL_NAMES = ['gpt-4v',
               'gpt-4o',
               'llava-v1.5-7b',
               'llava-v1.5-13b',
               'cogvlm-chat',
               'blip-flan-xl',
               'blip-flan-xxl']

DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
RESULT_DIRS = ['/projects/ml4science/VLM4Bio/']
EVAL_TYPE = ['prompting_results']
TASK_TYPE= ['direct', 'selection']
start_dir = os.path.join(RESULT_DIRS[0], EVAL_TYPE[0])

## Fish

In [6]:
print('######### Contextual ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'fish-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'contextual'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


print('######### Dense-caption ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'fish-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'dense-caption'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


print('######### CoT ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'fish-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'cot'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


print('######### No-prompting ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'fish-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'no-prompting'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


######### Contextual ###########
Writing evalutations for fish-prompting dataset and classification task and fish organism
Starting Evaluations for model gpt-4v on 500 results.
Evaluation completed for gpt-4v.
Starting Evaluations for model gpt-4o on 500 results.
Evaluation completed for gpt-4o.
Starting Evaluations for model llava-v1.5-7b on 500 results.
Evaluation completed for llava-v1.5-7b.
Starting Evaluations for model llava-v1.5-13b on 500 results.
Evaluation completed for llava-v1.5-13b.
Starting Evaluations for model cogvlm-chat on 500 results.
Evaluation completed for cogvlm-chat.
Starting Evaluations for model blip-flan-xl on 500 results.
Evaluation completed for blip-flan-xl.
Starting Evaluations for model blip-flan-xxl on 500 results.
Evaluation completed for blip-flan-xxl.
######### Dense-caption ###########
Writing evalutations for fish-prompting dataset and classification task and fish organism
Starting Evaluations for model gpt-4v on 500 results.
Evaluation completed f

In [7]:
print("\t\t\t\t\t###### No-Prompting ######")
df_eval = classification_tables('./tables/classification_fish-prompting_no-prompting_selection_fish.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

print("\t\t\t\t\t###### Contextual ######")
df_eval = classification_tables('./tables/classification_fish-prompting_contextual_selection_fish.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

print("\t\t\t\t\t###### Dense-Caption ######")
df_eval = classification_tables('./tables/classification_fish-prompting_dense-caption_selection_fish.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

print("\t\t\t\t\t###### CoT ######")
df_eval = classification_tables('./tables/classification_fish-prompting_cot_selection_fish.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

					###### No-Prompting ######


,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,34.40,5.60,45.20,14.80
1,gpt-4o,selection,79.00,12.60,0.00,8.40
2,llava-v1.5-7b,selection,41.60,8.80,0.00,49.60
3,llava-v1.5-13b,selection,35.40,8.20,0.00,56.40
4,cogvlm-chat,selection,31.00,11.80,0.00,57.20
5,blip-flan-xl,selection,28.60,0.00,0.00,71.40
6,blip-flan-xxl,selection,22.60,0.20,0.00,77.20


,correct_count
0,34.40
1,79.00
2,41.60
3,35.40
4,31.00
5,28.60
6,22.60


					###### Contextual ######


,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,30.00,3.20,59.20,7.60
1,gpt-4o,selection,77.20,14.40,0.00,8.40
2,llava-v1.5-7b,selection,40.20,10.80,0.00,49.00
3,llava-v1.5-13b,selection,35.60,10.20,0.00,54.20
4,cogvlm-chat,selection,25.60,14.20,0.00,60.20
5,blip-flan-xl,selection,27.20,0.00,0.00,72.80
6,blip-flan-xxl,selection,26.60,0.00,0.00,73.40


...

,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,18.80,0.80,79.40,1.00
1,gpt-4o,selection,78.60,13.60,0.00,7.80
2,llava-v1.5-7b,selection,26.00,8.80,0.00,65.20
3,llava-v1.5-13b,selection,27.60,11.20,0.00,61.20
4,cogvlm-chat,selection,32.00,12.20,0.00,55.80
5,blip-flan-xl,selection,28.40,0.00,0.00,71.60
6,blip-flan-xxl,selection,29.80,0.00,0.00,70.20


,correct_count
0,18.80
1,78.60
2,26.00
3,27.60
4,32.00
5,28.40
6,29.80


					###### CoT ######


,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,42.60,4.00,46.40,7.00
1,gpt-4o,selection,86.00,8.00,0.00,6.00
2,llava-v1.5-7b,selection,41.40,11.80,0.00,46.80
3,llava-v1.5-13b,selection,34.80,8.80,0.00,56.40
4,cogvlm-chat,selection,26.80,11.40,0.80,61.00
5,blip-flan-xl,selection,29.20,1.60,0.00,69.20
6,blip-flan-xxl,selection,24.60,0.00,0.00,75.40


,correct_count
0,42.60
1,86.00
2,41.40
3,34.80
4,26.80
5,29.20
6,24.60


## Bird

In [8]:
print('######### Contextual ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'bird-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'contextual'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


print('######### Dense-caption ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'bird-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'dense-caption'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


print('######### CoT ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'bird-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'cot'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


print('######### No-prompting ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'bird-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'no-prompting'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


######### Contextual ###########
Writing evalutations for bird-prompting dataset and classification task and bird organism
Starting Evaluations for model gpt-4v on 500 results.
Evaluation completed for gpt-4v.
Starting Evaluations for model gpt-4o on 500 results.
Evaluation completed for gpt-4o.
Starting Evaluations for model llava-v1.5-7b on 500 results.
Evaluation completed for llava-v1.5-7b.
Starting Evaluations for model llava-v1.5-13b on 500 results.
Evaluation completed for llava-v1.5-13b.
Starting Evaluations for model cogvlm-chat on 500 results.
Evaluation completed for cogvlm-chat.
Starting Evaluations for model blip-flan-xl on 500 results.
Evaluation completed for blip-flan-xl.
Starting Evaluations for model blip-flan-xxl on 500 results.
Evaluation completed for blip-flan-xxl.
######### Dense-caption ###########
Writing evalutations for bird-prompting dataset and classification task and bird organism
Starting Evaluations for model gpt-4v on 500 results.
Evaluation completed f

In [10]:
print("\t\t\t\t\t###### No-Prompting ######")
df_eval = classification_tables('./tables/classification_bird-prompting_no-prompting_selection_bird.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

print("\t\t\t\t\t###### Contextual ######")
df_eval = classification_tables('./tables/classification_bird-prompting_contextual_selection_bird.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

print("\t\t\t\t\t###### Dense-Caption ######")
df_eval = classification_tables('./tables/classification_bird-prompting_dense-caption_selection_bird.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

print("\t\t\t\t\t###### CoT ######")
df_eval = classification_tables('./tables/classification_bird-prompting_cot_selection_bird.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

					###### No-Prompting ######


,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,78.80,1.40,16.80,3.00
1,gpt-4o,selection,97.60,1.20,0.00,1.20
2,llava-v1.5-7b,selection,44.20,2.60,0.00,53.20
3,llava-v1.5-13b,selection,49.80,2.60,0.00,47.60
4,cogvlm-chat,selection,45.40,2.60,0.00,52.00
5,blip-flan-xl,selection,35.60,0.00,0.00,64.40
6,blip-flan-xxl,selection,35.80,0.00,0.00,64.20


,correct_count
0,78.80
1,97.60
2,44.20
3,49.80
4,45.40
5,35.60
6,35.80


					###### Contextual ######


,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,78.60,2.80,14.60,4.00
1,gpt-4o,selection,98.60,0.60,0.20,0.60
2,llava-v1.5-7b,selection,44.00,1.80,0.00,54.20
3,llava-v1.5-13b,selection,52.00,3.80,0.00,44.20
4,cogvlm-chat,selection,49.40,2.80,0.00,47.80
5,blip-flan-xl,selection,35.60,0.00,0.00,64.40
6,blip-flan-xxl,selection,30.40,0.20,0.00,69.40


...

,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,87.40,1.00,8.40,3.20
1,gpt-4o,selection,97.00,1.20,0.40,1.40
2,llava-v1.5-7b,selection,33.40,1.80,0.00,64.80
3,llava-v1.5-13b,selection,41.00,2.20,0.00,56.80
4,cogvlm-chat,selection,44.00,2.00,0.00,54.00
5,blip-flan-xl,selection,25.60,0.00,0.00,74.40
6,blip-flan-xxl,selection,22.80,0.00,0.00,77.20


,correct_count
0,87.40
1,97.00
2,33.40
3,41.00
4,44.00
5,25.60
6,22.80


					###### CoT ######


,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,62.60,0.60,36.20,0.60
1,gpt-4o,selection,98.60,0.40,0.40,0.60
2,llava-v1.5-7b,selection,37.40,2.20,0.00,60.40
3,llava-v1.5-13b,selection,47.80,3.00,0.00,49.20
4,cogvlm-chat,selection,42.20,2.60,0.20,55.00
5,blip-flan-xl,selection,30.60,0.60,0.00,68.80
6,blip-flan-xxl,selection,31.00,0.00,0.00,69.00


,correct_count
0,62.60
1,98.60
2,37.40
3,47.80
4,42.20
5,30.60
6,31.00


## Butterfly

In [11]:
print('######### Contextual ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'butterfly-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'contextual'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


print('######### Dense-caption ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'butterfly-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'dense-caption'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


print('######### CoT ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'butterfly-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'cot'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


print('######### No-prompting ###########')

task = 'classification'

# DATASETS = ['fish-prompting', 'bird-prompting', 'butterfly-prompting']
dataset = 'butterfly-prompting'

# ['contextual', 'dense-caption', 'cot', 'no-prompting']
prompting_technique = 'no-prompting'

organism = None 
if 'fish' in dataset:
    organism = 'fish'
elif 'bird' in dataset:
    organism = 'bird'
elif 'butterfly' in dataset:
    organism = 'butterfly'

print(f'Writing evalutations for {dataset} dataset and {task} task and {organism} organism')

task_type = TASK_TYPE[1] # 0: direct, 1: selection
write_evaluation(dataset=dataset, task_type=task_type, task=task, start_dir=start_dir, organism=organism, prompting_technique=prompting_technique)


######### Contextual ###########
Writing evalutations for butterfly-prompting dataset and classification task and butterfly organism
Starting Evaluations for model gpt-4v on 500 results.
Evaluation completed for gpt-4v.
Starting Evaluations for model gpt-4o on 500 results.
Evaluation completed for gpt-4o.
Starting Evaluations for model llava-v1.5-7b on 500 results.
Evaluation completed for llava-v1.5-7b.
Starting Evaluations for model llava-v1.5-13b on 500 results.
Evaluation completed for llava-v1.5-13b.
Starting Evaluations for model cogvlm-chat on 500 results.
Evaluation completed for cogvlm-chat.
Starting Evaluations for model blip-flan-xl on 500 results.
Evaluation completed for blip-flan-xl.
Starting Evaluations for model blip-flan-xxl on 500 results.
Evaluation completed for blip-flan-xxl.
######### Dense-caption ###########
Writing evalutations for butterfly-prompting dataset and classification task and butterfly organism
Starting Evaluations for model gpt-4v on 500 results.
Ev

In [13]:
print("\t\t\t\t\t###### No-Prompting ######")
df_eval = classification_tables('./tables/classification_butterfly-prompting_no-prompting_selection_butterfly.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

print("\t\t\t\t\t###### Contextual ######")
df_eval = classification_tables('./tables/classification_butterfly-prompting_contextual_selection_butterfly.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

print("\t\t\t\t\t###### Dense-Caption ######")
df_eval = classification_tables('./tables/classification_butterfly-prompting_dense-caption_selection_butterfly.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

print("\t\t\t\t\t###### CoT ######")
df_eval = classification_tables('./tables/classification_butterfly-prompting_cot_selection_butterfly.jsonl', model_lists=PROMPT_MODELS)
display(df_eval)

					###### No-Prompting ######


,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,13.20,12.60,65.60,8.60
1,gpt-4o,selection,56.40,24.00,0.20,19.40
2,llava-v1.5-7b,selection,27.20,29.40,0.00,43.40
3,llava-v1.5-13b,selection,26.80,30.00,0.00,43.20
4,cogvlm-chat,selection,25.60,34.80,0.00,39.60
5,blip-flan-xl,selection,24.40,0.00,0.00,75.60
6,blip-flan-xxl,selection,21.20,0.00,0.00,78.80


,correct_count
0,13.20
1,56.40
2,27.20
3,26.80
4,25.60
5,24.40
6,21.20


					###### Contextual ######


,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,9.20,7.60,77.60,5.60
1,gpt-4o,selection,56.20,24.80,0.40,18.60
2,llava-v1.5-7b,selection,26.00,31.40,0.00,42.60
3,llava-v1.5-13b,selection,24.60,29.20,0.00,46.20
4,cogvlm-chat,selection,27.20,31.00,0.00,41.80
5,blip-flan-xl,selection,23.60,0.00,0.00,76.40
6,blip-flan-xxl,selection,24.60,0.00,0.00,75.40


...

,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,49.60,14.20,24.60,11.60
1,gpt-4o,selection,63.20,21.20,0.00,15.60
2,llava-v1.5-7b,selection,25.20,26.40,0.00,48.40
3,llava-v1.5-13b,selection,23.80,25.80,0.00,50.40
4,cogvlm-chat,selection,27.00,31.00,0.00,42.00
5,blip-flan-xl,selection,23.20,0.00,0.00,76.80
6,blip-flan-xxl,selection,23.20,0.00,0.00,76.80


,correct_count
0,49.60
1,63.20
2,25.20
3,23.80
4,27.00
5,23.20
6,23.20


					###### CoT ######


,Models,Question-type,correct_count,correct_ge_count,NA_count,incorrect_count
0,gpt-4v,selection,63.60,11.80,16.40,8.20
1,gpt-4o,selection,74.60,16.20,0.20,9.00
2,llava-v1.5-7b,selection,21.40,34.80,0.00,43.80
3,llava-v1.5-13b,selection,23.20,28.20,0.00,48.60
4,cogvlm-chat,selection,34.60,30.00,0.00,35.40
5,blip-flan-xl,selection,37.20,16.80,0.00,46.00
6,blip-flan-xxl,selection,23.60,0.20,0.00,76.20


,correct_count
0,63.60
1,74.60
2,21.40
3,23.20
4,34.60
5,37.20
6,23.60
